In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import os
from tqdm import tqdm



In [2]:
# ----------------------------
# Device
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# Paths
# ----------------------------
dataset_path = r"D:\Users\Lenova\Desktop\YukTha\dataset"
train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "val")

# ----------------------------



In [13]:
# Data Augmentation (Prevents Overfitting)
# ----------------------------
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),

    # 🔥 New Robust Augmentations
    transforms.ColorJitter(
        brightness=0.3,
        contrast=0.3,
        saturation=0.3,
        hue=0.1
    ),

    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.GaussianBlur(kernel_size=3),

    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Class mapping:", train_dataset.class_to_idx)

#

Class mapping: {'ai': 0, 'real': 1}


In [ ]:
model = models.mobilenet_v2(weights=None)

model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.last_channel, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 2)
)

MODEL_PATH = r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v2.pth"

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)

print("✅ v2 model loaded successfully")


✅ v2 model loaded successfully


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)


In [36]:
MODEL_PATH = r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v3.pth"

print("Loading model from:", MODEL_PATH)


Loading model from: D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v3.pth


In [37]:
# 1. Load model

#v4
model = models.mobilenet_v2(weights=None)

model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.last_channel, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 2)
)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)

print("Model loaded")


Model loaded


In [38]:
# Freeze early layers (keep high-level features stable)
for param in model.features[:10].parameters():
    param.requires_grad = False


In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5
)


In [40]:
epochs = 3

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    print(f"\nEpoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {running_loss:.4f}")
    print(f"Train Acc: {train_acc:.4f}")
    print(f"Val Acc: {val_acc:.4f}")



Epoch [1/3]
Train Loss: 32.0159
Train Acc: 0.9330
Val Acc: 0.9583

Epoch [2/3]
Train Loss: 35.0952
Train Acc: 0.9283
Val Acc: 0.9467

Epoch [3/3]
Train Loss: 32.8183
Train Acc: 0.9313
Val Acc: 0.9450


In [41]:
SAVE_PATH = r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v4.pth"
torch.save(model.state_dict(), SAVE_PATH)

print("🔥 v4 model saved successfully")



🔥 v4 model saved successfully


In [43]:
print(train_dataset.class_to_idx)


{'ai': 0, 'real': 1}


In [44]:
pip install lime scikit-image


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
    --------------------------------------- 0.3/11.9 MB ? eta -:--:--
    --------------------------------------- 0.3/11.9 MB ? eta -:--:--
    --------------------------------------- 0.3/11.9 MB ? eta -:--:--
    --------------------------------------- 0.3/11.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.9 MB 285.6 kB/s eta 0:00:40
   - --------------------------

In [35]:
epochs = 3

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total

    print(f"\nEpoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {running_loss:.4f}")
    print(f"Train Acc: {train_acc:.4f}")
    print(f"Val Acc: {val_acc:.4f}")


KeyboardInterrupt: 

In [21]:
SAVE_PATH = r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v3.pth"
torch.save(model.state_dict(), SAVE_PATH)

print("🔥 v3 model saved successfully")



🔥 v3 model saved successfully


In [ ]:
model.eval()
val_correct = 0
val_total = 0

with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

val_acc = val_correct / val_total

print(f"\nEpoch [{epoch+1}/{epochs}]")
print(f"Train Loss: {train_loss:.4f}")
print(f"Train Acc: {train_acc:.4f}")
print(f"Val Acc: {val_acc:.4f}")



Epoch [6/6]
Train Loss: 22.1334
Train Acc: 0.9597
Val Acc: 0.9567


In [ ]:
if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(),
                   r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v3.pth")
        print("Best model saved!")
        trigger_times = 0
else:
        trigger_times += 1
        print(f"No improvement. Patience: {trigger_times}/{patience}")

        if trigger_times >= patience:
            print("Early stopping triggered.")
             

print("\nTraining Complete.")
print("Best Validation Accuracy:", best_val_acc)

Best model saved!

Training Complete.
Best Validation Accuracy: 0.9566666666666667


In [22]:
print(train_dataset.class_to_idx)


{'ai': 0, 'real': 1}


In [11]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import torch

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Detailed Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=["Real", "AI"]))


Confusion Matrix:
[[297   3]
 [ 23 277]]

Classification Report:
              precision    recall  f1-score   support

        Real       0.93      0.99      0.96       300
          AI       0.99      0.92      0.96       300

    accuracy                           0.96       600
   macro avg       0.96      0.96      0.96       600
weighted avg       0.96      0.96      0.96       600



In [12]:
print(train_dataset.class_to_idx)


{'ai': 0, 'real': 1}


In [ ]:
SAVE_PATH = r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model_v3.pth"
torch.save(model.state_dict(), SAVE_PATH)

print("🔥 v3 model saved successfully")
